# Reading in Text

In this section of code I import necessary packages and query the convention database.

In [1]:
import nltk
import sqlite3
import spacy
from subject_object_extraction import findSVOs
from collections import Counter, defaultdict
from string import punctuation
from nltk.corpus import stopwords
nltk.download('stopwords')
sw = stopwords.words('english')
parser = spacy.load('en_core_web_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aidan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
dem_list = list()
rep_list = list()

convention_db = sqlite3.connect("ConventionSpeeches.db")
convention_cur = convention_db.cursor()

query_results = convention_cur.execute(
                            '''
                                SELECT text, party
                                FROM conventions
                                WHERE speaker != "Unknown"
                               
                                
                            ''')


convention_data = defaultdict(str)

for row in query_results :
    text, party = row

    # A nice trick to get rid of punctuation
    text = "".join([w for w in text if w not in punctuation])    
    text = [w.lower() for w in text.split() if w.isalpha()]
    
    convention_data[party] += " ".join(text) + " "
    


# Strange Fixes

The text in conventionspeeches was way too big, so I made a for loop with help from the internet to cut it down to a workable amount of characters.

In [37]:
dem_split = convention_data["Democratic"]
rep_split = convention_data["Republican"]
indexCount = 0
index2Count = 0

demo_string = str()
repub_string = str()

#https://www.tutorialspoint.com/How-to-get-first-100-characters-of-the-string-in-Python
for character in dem_split:
    indexCount +=1

    if indexCount <=1000000:
     
      demo_string += character
    else:
        break
for character in rep_split:
    index2Count +=1

    if index2Count <=1000000:

        repub_string += character
    else:
        break
print(len(demo_string))
print(len(repub_string))


1000000
1000000


# Parsing

Here I do all the parsing work.

In [35]:
parsed_repub = parser(repub_string)
parsed_democrat = parser(demo_string)
democrat_svo = (findSVOs(parsed_democrat))
rep_svo = (findSVOs(parsed_repub))
count_dem = Counter(democrat_svo)
count_rep = Counter(rep_svo)

# Results

The most common SVOs for republicans seem to focus more on God. Republicans ask God to bless you, the states, and America quite a bit more frequently. These SVOs also seem to have a celebratory tone, with SVOs such as "we,did,it", "i,thank,you", and "we,have,president". Republicans also seem to focus more on "you" and the president than the democrats.

In [31]:
count_rep.most_common(20)

[(('you', 'thank', 'you'), 67),
 (('i', 'tell', 'you'), 52),
 (('god', 'bless', 'you'), 51),
 (('god', 'bless', 'america'), 42),
 (('god', 'bless', 'states'), 24),
 (('we', 'need', 'president'), 18),
 (('me', 'tell', 'you'), 15),
 (('i', 'thank', 'you'), 11),
 (('we', 'did', 'it'), 11),
 (('i', 'talk', 'you'), 10),
 (('we', 'have', 'president'), 9),
 (('they', 'get', 'it'), 9),
 (('we', 'thank', 'you'), 8),
 (('we', 'need', 'leader'), 8),
 (('we', 'cut', 'taxes'), 8),
 (('i', 'stand', 'you'), 8),
 (('i', 'had', 'privilege'), 8),
 (('he', 'do', 'it'), 8),
 (('he', 'told', 'me'), 7),
 (('i', 'speak', 'you'), 7)]

The democrats talk about Obama quite a bit, while the Republicans do not even have him in the top 20 SVOs. There also seems to be more talk specific to policy in the top SVOs for the democratic party. Honestly, there aren't too many differences in the SVOs that I find very interesting. My favorite is that both parties take credit for tax cuts.

In [32]:
count_dem.most_common(20)

[(('i', 'tell', 'you'), 33),
 (('we', 'need', 'president'), 32),
 (('you', 'thank', 'you'), 31),
 (('god', 'bless', 'you'), 31),
 (('god', 'bless', 'america'), 26),
 (('we', 'do', 'it'), 24),
 (('me', 'tell', 'you'), 22),
 (('i', 'seen', 'it'), 17),
 (('we', 'elect', 'president'), 13),
 (('we', 'have', 'president'), 13),
 (('we', '!afford', 'years'), 13),
 (('we', '!afford', 'more'), 13),
 (('god', 'bless', 'states'), 11),
 (('we', 'reelect', 'president'), 10),
 (('president', 'barack', 'obama'), 10),
 (('i', 'stand', 'you'), 10),
 (('we', 'need', 'change'), 10),
 (('he', 'cut', 'taxes'), 9),
 (('we', 'elect', 'obama'), 9),
 (('obama', 'took', 'office'), 8)]